In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.color import rgb2gray
from skimage.filters import gabor
from skimage.measure import shannon_entropy
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
from skimage.feature import hog
from scipy import stats


# Load the trained model
model = tf.keras.models.load_model("tree_detection_model.h5")

# Image enhancement function
def enhance_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    return clahe.apply(gray)

# Feature extraction function (same as in training)
def extract_texture_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128))

    gray = rgb2gray(image)
    gray = (gray * 255).astype(np.uint8)

    enhanced_gray = enhance_image(image)

    gabor_kernels = [cv2.getGaborKernel((5, 5), sigma, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
                     for sigma in [1, 3] for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]]
    gabor_features = [np.mean(cv2.filter2D(enhanced_gray, cv2.CV_8UC3, k)) for k in gabor_kernels]

    glcm = graycomatrix(enhanced_gray, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
    glcm_features = [graycoprops(glcm, prop).mean() for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']]

    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(11), density=True)

    hog_features = hog(gray, orientations=6, pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)

    ent = shannon_entropy(enhanced_gray)
    mean, variance, skewness, kurtosis = np.mean(enhanced_gray), np.var(enhanced_gray), stats.skew(enhanced_gray.flatten()), stats.kurtosis(enhanced_gray.flatten())

    sift = cv2.SIFT_create()
    keypoints, _ = sift.detectAndCompute(enhanced_gray, None)
    sift_count = len(keypoints)

    combined_features = np.hstack((glcm_features, lbp_hist, hog_features, gabor_features, [ent, mean, variance, skewness, kurtosis, sift_count]))
    return combined_features

# Prediction function
def predict_images_from_folder(folder_path):
    image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.png', '.jpg', '.jpeg','.tif'))]
    
    predictions = []
    for img_path in image_paths:
        # Load image for CNN
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (128, 128))
        image_array = np.expand_dims(image, axis=0).astype(np.float32) / 255.0  # Normalize

        # Extract handcrafted features
        manual_features = extract_texture_features(img_path)
        manual_features_array = np.expand_dims(manual_features, axis=0)

        # Predict
        pred = model.predict([image_array, manual_features_array])[0][0]
        label = "Tree" if pred > 0.5 else "Non-Tree"

        predictions.append((img_path, label, pred))

    return predictions

# Run prediction on test folder
test_folder = r"C:\Users\Admin\Desktop\binary classification\img"
results = predict_images_from_folder(test_folder)

# Print results
for img_path, label, confidence in results:
    print(f"Image: {img_path} --> Prediction: {label} (Confidence: {confidence:.4f})")